## 检查井曲线数据步长


In [ ]:
"""
检查所有LAS文件中曲线数据的步长一致性
"""

import os
from pathlib import Path

import lasio
import numpy as np
import pandas as pd


In [ ]:
# ============ 1. 读取所有LAS文件并检查基本信息 ============
las_dir = Path("..") / "data" / "vertical_well_common_las"

las_files = list(las_dir.glob("*.las")) + list(las_dir.glob("*.LAS"))
las_files = list(set(las_files))
las_files.sort()

print(f"找到 {len(las_files)} 个 LAS 文件\n")
print("=" * 100)

# 存储每个井的详细信息
well_info = []

for las_file in las_files:
    try:
        las = lasio.read(las_file)
        well_name = las_file.stem

        # 获取深度信息
        depth = las.index
        n_samples = len(depth)

        # 计算深度步长
        depth_diffs = np.diff(depth)

        # 统计步长信息
        min_step = np.min(depth_diffs)
        max_step = np.max(depth_diffs)
        mean_step = np.mean(depth_diffs)
        std_step = np.std(depth_diffs)

        # 检查步长是否一致（标准差很小说明一致）
        is_uniform = std_step < 0.001  # 步长标准差小于0.001m认为是均匀的

        well_info.append(
            {
                "well_name": well_name,
                "n_samples": n_samples,
                "depth_start": depth[0],
                "depth_end": depth[-1],
                "depth_range": depth[-1] - depth[0],
                "min_step": min_step,
                "max_step": max_step,
                "mean_step": mean_step,
                "std_step": std_step,
                "is_uniform": is_uniform,
                "n_curves": len(las.keys()),
            }
        )

        status = "✓ 均匀" if is_uniform else "✗ 不均匀"
        print(f"{well_name:15s} | 样本数: {n_samples:5d} | 步长: {mean_step:.4f}±{std_step:.4f}m | {status}")

    except Exception as e:
        print(f"✗ 读取 {las_file.name} 失败: {e}")

print("=" * 100)


In [ ]:
# ============ 2. 创建汇总表格 ============
df_summary = pd.DataFrame(well_info)

print("\n" + "=" * 100)
print("汇总统计:\n")
print(df_summary.to_string(index=False))
print("\n" + "=" * 100)
